# Credits for bubarino giving me the huggingface import code (感谢 bubarino 给了我 huggingface 导入代码)

In [1]:
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS
!apt-get update && apt-get install -y --no-install-recommends tzdata ffmpeg libsox-dev parallel aria2 git git-lfs && git lfs install
!pip install -r requirements.txt

Cloning into 'GPT-SoVITS'...
remote: Enumerating objects: 2829, done.
remote: Counting objects: 100% (745/745), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 2829 (delta 621), reused 563 (delta 538), pack-reused 2084
Receiving objects: 100% (2829/2829), 6.47 MiB | 1.64 MiB/s, done.
Resolving deltas: 100% (1591/1591), done.
/home/xiaolt1/workspace/ai_learn/GPT-SoVITS-main/GPT-SoVITS


/home/xiaolt1/anaconda3/envs/learn/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 591.4 kB/s eta 0:00:001m483.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:01:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.1 MB/s eta 0:00:002.3 MB/s eta 0:00:01
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00

In [ ]:
# @title Download pretrained models 下载预训练模型
!mkdir -p /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!mkdir -p /content/GPT-SoVITS/tools/damo_asr/models
!mkdir -p /content/GPT-SoVITS/tools/uvr5
%cd /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
%cd /content/GPT-SoVITS/tools/damo_asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
# @title UVR5 pretrains 安装uvr5模型
%cd /content/GPT-SoVITS/tools/uvr5
!git clone https://huggingface.co/Delik/uvr5_weights
!git config core.sparseCheckout true
!mv /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/

In [ ]:
#@title Create folder models 创建文件夹模型
import os
base_directory = "/content/GPT-SoVITS"
folder_names = ["SoVITS_weights", "GPT_weights"]

for folder_name in folder_names:
  if os.path.exists(os.path.join(base_directory, folder_name)):
    print(f"The folder '{folder_name}' already exists. (文件夹'{folder_name}'已经存在。)")
  else:
    os.makedirs(os.path.join(base_directory, folder_name))
    print(f"The folder '{folder_name}' was created successfully! (文件夹'{folder_name}'已成功创建！)")

print("All folders have been created. (所有文件夹均已创建。)")

In [ ]:
import requests
import zipfile
import shutil
import os

#@title Import model 导入模型 (HuggingFace)
hf_link = 'https://huggingface.co/modelloosrvcc/Nagisa_Shingetsu_GPT-SoVITS/resolve/main/Nagisa.zip' #@param {type: "string"}

output_path = '/content/'

response = requests.get(hf_link)
with open(output_path + 'file.zip', 'wb') as file:
    file.write(response.content)

with zipfile.ZipFile(output_path + 'file.zip', 'r') as zip_ref:
    zip_ref.extractall(output_path)

os.remove(output_path + "file.zip")

source_directory = output_path
SoVITS_destination_directory = '/content/GPT-SoVITS/SoVITS_weights'
GPT_destination_directory = '/content/GPT-SoVITS/GPT_weights'

for filename in os.listdir(source_directory):
    if filename.endswith(".pth"):
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(SoVITS_destination_directory, filename)
        shutil.move(source_path, destination_path)

for filename in os.listdir(source_directory):
    if filename.endswith(".ckpt"):
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(GPT_destination_directory, filename)
        shutil.move(source_path, destination_path)

print(f'Model downloaded. (模型已下载。)')

In [ ]:
# @title launch WebUI 启动WebUI
!/usr/local/bin/pip install ipykernel
!sed -i '10s/False/True/' /content/GPT-SoVITS/config.py
%cd /content/GPT-SoVITS/
!/usr/local/bin/python  webui.py